# ANÁLISIS DE NIVELES DE SEVERIDAD EN SINIESTRALIDAD POR ACTOR Y SERIES DE TIEMPO PARA LA CIUDAD DE BOGOTÁ.
---------------------
La siniestralidad vial es una problemática de salud pública que genera más de un millón de muertes al año a nivel mundial, en el caso de la ciudad de Bogotá, Colombia, significó 536 fatalidades para el año 2022, razón por la cual es importante el estudio sistemático para entender la siniestralidad vial, así como de la severidad de estos eventos. Con el desarrollo de algoritmos programables de aprendizaje de máquina, se ha comenzado a emplear diferentes algoritmos para estudiar la ocurrencia de siniestros y su severidad desde un enfoque estadístico y de la ciencia de datos..

> *Teniendo en cuenta lo anterior, la presente investigación tiene como objetivo la aplicación y comparación de modelos de clasificación supervisados como lo son, regresión logística (LOGREG), k vecinos cercanos (KNN), bosques aleatorios (RF), máquinas de soporte vectorial (SVM) y red neuronal multicapa (MLP) en la predicción de los niveles de severidad en los siniestros viales de la ciudad de Bogotá*


Todos los datos han sido anonimizados, así como, se ha modificado la base de datos con fines académicos.

## I. PROBLEMA DE INVESTIGACIÓN
-------------

La siniestralidad vial se clasifica de acuerdo con la gravedad o el tipo de colisión, comúnmente se modela la frecuencia y la gravedad por separado; sin embargo, algunos investigadores han trabajado modelos de frecuencia para los diferentes tipos de gravedad de los siniestros viales; En los últimos años la comunidad científica ha adelantado estudios para el entendimiento de la severidad de los siniestros viales aplicando modelos no paramétricos y algoritmos basados en el aprendizaje de máquina.
De esta manera, el código presenta la aplicación de técnicas de ML orientadas a la predicción de la severidad de los siniestros viales en la ciudad de Bogotá, esta metodología permite el diseño de herramientas y medidas orientadas a la mitigación de altos niveles de severidad.



## II.	IMPORTACIÓN DE LIBRERÍAS
------------------------------
Dentro de los elementos básicos en el análisis en Python es la correcta importación y cargue de las librerías.

In [2]:
# Actualizamos scikit-learn a la última versión
%pip install -U scikit-learn
%pip install -U pandas
%pip install -U seaborn
%pip install -U numpy
%pip install -U matplotlib
%pip install -U datetime
%pip install -U scipy
%pip install -U statsmodel
%pip install -U statsmodels
%pip install -U plotly
%pip install -U mlxtend


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

import sklearn
# Otras librerías de utilidad
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
import warnings
from scipy import stats
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
from scipy.stats import wilcoxon
import plotly.graph_objects as go

# Ignorar advertencias del código.
import warnings
warnings.filterwarnings('ignore')

# Las figuras en formato retina se ven más limpias
%config InlineBackend.figure_format = 'retina'

# Configuraciones de Matplotlib
%matplotlib inline
plt.rcParams['figure.dpi'] = 250
mpl.rcParams['figure.figsize'] = (9, 7)

# Estilo por defecto para notebooks.
sns.set_theme('notebook')

C:\Users\jagarcia\AppData\Local\Temp\ipykernel_16016\3459544716.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 1. FUNCIONES DE UTILIDAD
----------

El siguiente apartado muestra algunas de las funciones importantes para entender el próximo desarrollo de las funciones.

In [ ]:
# Gracicar la curva de aprendizaje de determinada métrica de agrupamiento.

def plot_metric(K, scores, metric_name):
  plt.figure(dpi=110, figsize=(9, 5))
  plt.plot(K, scores, 'bx-')
  plt.xticks(K); plt.xlabel('$k$', fontdict=dict(family = 'serif', size = 14));  plt.ylabel(metric_name, fontdict=dict(family = 'serif', size = 14));
  plt.title(f'K vs {metric_name}', fontdict=dict(family = 'serif', size = 18))
  plt.show()

In [ ]:
from mlxtend.plotting import plot_decision_regions

# Función para visualizar la superficie de decisión de un clasificador.
def plot_decision_region(X, y, clf, classes, title = ""):
  fig, ax = plt.subplots(dpi = 120)
  plot_decision_regions(X, y, clf = clf, ax = ax)
  handles, _ = ax.get_legend_handles_labels()
  ax.legend(handles, classes)
  ax.set_title(title)
  fig.show()

In [ ]:
def list_confusion_matrix(cm,classes):
  df = pd.DataFrame(data = cm,
                    index = pd.MultiIndex.from_product([['Valor real'], classes]),
                    columns = pd.MultiIndex.from_product([['Valor predicho'], classes]))

  return df

In [ ]:
# Función para evaluar en conjunto diferentes medidas de tendencia central y estadísticos de descriptivos.
def describe2(df, stats):
  d = (df.describe().round(2)).iloc[:,0:]
  return d.append(df.reindex(d.columns, axis = 1).agg(stats)).round(2)

In [ ]:
# Gráficar la predicción de los datos de temperatura.

def plot_prediction(params, ys, test_date_index):

  train_data = serie_dia.loc[:test_date_index[0]]
  _y_test, _y_forward, _y_last = ys
  # Graficamos los valores predichos.
  fig = go.Figure(layout = dict(
       title = f'<b>Siniestros viales en Bogota (01/10/22 - 14/10/23)</b> <br> {params}',
       dragmode= 'pan', width = 1200, height = 600))

  fig.add_trace(go.Scatter(x = train_data.index,  # Datos originales hasta la primer semana predicha. (fechas)
                          y = train_data.values, # Datos originales hasta la primer semana predicha. (temperaturas)
                          mode = 'lines',
                          name = 'Valores de entrenamiento y pruebas'))

  #Gráfica de los valores de prueba reales.
  fig.add_trace(go.Scatter(x = test_date_index,
                          y = _y_test,
                          mode='lines+markers',
                          name='Valores reales (y)'))


  #Gráfica de los valores predichos a partir de las ventanas de X_test.
  fig.add_trace(go.Scatter(x = test_date_index,
                          y = _y_forward,
                          mode = 'lines+markers',
                          name = 'Valores predichos a partir de datos reales'))

  #Gráfica de los valores predichos a partir de ventanas creadas proceduralmente.
  fig.add_trace(go.Scatter(x = test_date_index,
                          y = _y_last,
                          mode='lines+markers',
                          name='Valores predichos a partir de datos predichos'))

  fig.show(config = dict({'scrollZoom': True}))

In [ ]:
# Gráfica de la varianza explicada acumulada.

def cumulative_explained_variance_plot(expl_variance):

  cum_var_exp = np.cumsum(expl_variance)

  plt.figure(dpi = 100, figsize = (8, 6))
  plt.title('Curva acumulativa de la varianza explicada VS n° de componentes principales',
            fontdict= dict(family ='serif', size = 16))
  plt.xlabel('Número de componentes principales',
             fontdict= dict(family ='serif', size = 14))
  plt.ylabel('Varianza explicada acumulativa',
             fontdict= dict(family ='serif', size = 14))

  nc = np.arange(1, expl_variance.shape[0] + 1)

  plt.plot(nc, cum_var_exp, '--r')
  plt.plot(nc, cum_var_exp, 'c*', ms = 5)
  plt.show()

## III PREPARACION Y PREPROCESAMIENTO
--------------
El trabajo se encuentra dividido en dos apartados, el primero de estos se enfoca en el análisis de la siniestralidad y los actores viales, así bien, a partir de la información del SIMIT (Sistema de Información de Accidentes de Tránsito) de los actores viales se busca identificar los niveles de severidad de los diferentes siniestros registrados.

### 1.	CARGA DE DATOS
----------
En este apartado se carga la base de datos de los actores viales para los siniestros de 2021, para el respectivo análisis.

In [ ]:
#Cargue de la base de datos.
ACTORV_df = pd.read_csv('C:\\sqldeveloper\\sqldeveloper\\bin\\ACTOR_VIAL_SINIESTRO.csv',engine='python')
ACTORV_df['FECHA'] = pd.to_datetime(ACTORV_df['FECHA'])
ACTORV_df['EDAD'] = ACTORV_df['EDAD'].astype('Int64')
ACTORV_df['MOD_VEHICULO'] = ACTORV_df['MOD_VEHICULO'].astype('Int64')
ACTORV_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158838 entries, 0 to 158837
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   FORMULARIO         158838 non-null  object        
 1   CODIGO_ACCIDENTE   158838 non-null  int64         
 2   FECHA              158838 non-null  datetime64[ns]
 3   CODIGO_VICTIMA     158838 non-null  int64         
 4   CODIGO_VEHICULO    152820 non-null  float64       
 5   TIPO_ACTOR         158814 non-null  object        
 6   ESTADO             156046 non-null  object        
 7   GENERO             155133 non-null  object        
 8   EDAD               154415 non-null  Int64         
 9   GRAVEDAD           158838 non-null  int64         
 10  CLASE_SINIESTRO    158838 non-null  int64         
 11  DIRECCION          158838 non-null  object        
 12  LONGITUD           158838 non-null  float64       
 13  LATITUD            158838 non-null  float64 

### 2.	PREPARACIÓN DE BASES DE DATOS
------------------------
En la siguiente sección se realiza la limpieza de las diferentes bases de datos, y se aplican funciones para la selección condicional a partir del conocimiento de la problemática.

In [ ]:
#Identificación de los bici usuarios
ACTORV_df["MOD_VEHICULO"] = np.where(ACTORV_df["TIPO_ACTOR"] == "CICLISTA", 5000, ACTORV_df["MOD_VEHICULO"])
ACTORV_df["CLASE_VEHICULO"] = np.where(ACTORV_df["TIPO_ACTOR"] == "CICLISTA", 13, ACTORV_df["CLASE_VEHICULO"])
ACTORV_df["SERVICIO_VEHICULO"] = np.where(ACTORV_df["TIPO_ACTOR"] == "CICLISTA", 5, ACTORV_df["SERVICIO_VEHICULO"])

ACTORV_df["MOD_VEHICULO"] = np.where(ACTORV_df["TIPO_ACTOR"] == "PEATON", 3000, ACTORV_df["MOD_VEHICULO"])
ACTORV_df["CLASE_VEHICULO"] = np.where(ACTORV_df["TIPO_ACTOR"] == "PEATON", 30, ACTORV_df["CLASE_VEHICULO"])
ACTORV_df["SERVICIO_VEHICULO"] = np.where(ACTORV_df["TIPO_ACTOR"] == "PEATON", 6, ACTORV_df["SERVICIO_VEHICULO"])


In [ ]:
#Limpieza de la base de datos a partir de la información ml digitada.
ACTORV_df = ACTORV_df[      (ACTORV_df['GENERO'] != 'SIN INFORMACION') &
                            (ACTORV_df['TIPO_ACTOR'].notnull()) &
                            (ACTORV_df['GENERO'].notnull()) &
                            (ACTORV_df['ESTADO'].notnull()) &
                            (ACTORV_df['MOD_VEHICULO'].notnull()) &
                            (ACTORV_df['MOD_VEHICULO'] >= 1950) &
                            (ACTORV_df['CLASE_VEHICULO'].notnull()) &
                            (ACTORV_df['CLASE_VEHICULO'] != 0) &
                            (ACTORV_df['SERVICIO_VEHICULO'].notnull()) &
                            (ACTORV_df['CODIGO_CAUSA'].notnull()) &
                            (ACTORV_df['CODIGO_CAUSA'] != 0) &
                            (ACTORV_df['EDAD'] >= 5) &
                            (ACTORV_df['EDAD'] <= 85)
                                                                ]

In [ ]:
# Limpieza variables con menor cantidad de registros
counts_clase = ACTORV_df['CLASE_VEHICULO'].value_counts()
ACTORV_df = ACTORV_df[~ACTORV_df['CLASE_VEHICULO'].isin(counts_clase[counts_clase < 100].index)]
counts_servicio = ACTORV_df['SERVICIO_VEHICULO'].value_counts()
ACTORV_df = ACTORV_df[~ACTORV_df['SERVICIO_VEHICULO'].isin(counts_servicio[counts_servicio < 200].index)]
counts_siniestro = ACTORV_df['CLASE_SINIESTRO'].value_counts()
ACTORV_df = ACTORV_df[~ACTORV_df['CLASE_SINIESTRO'].isin(counts_siniestro[counts_siniestro < 750].index)]

In [ ]:
counts_siniestro

CLASE_SINIESTRO
1    134090
2      9648
4      3055
3      1521
6       517
5         6
Name: count, dtype: int64

In [ ]:
#Unión de causas en categorías de variables con menos cantidad de registros.
counts_causa = ACTORV_df['CODIGO_CAUSA'].value_counts()
ACTORV_df["CODIGO_CAUSA"] = np.where(ACTORV_df['CODIGO_CAUSA'].isin(counts_causa[counts_causa <= 1000].index), 1000, ACTORV_df["CODIGO_CAUSA"])

In [ ]:
#Discriminación de variables en grupos a partir del conocimiento.
ACTORV_df["EDAD_CAT"] =pd.cut(x=ACTORV_df['EDAD'], bins=[0,11,18,26,59,100])
ACTORV_df["MODELO_CAT"] =pd.cut(x=ACTORV_df['MOD_VEHICULO'], bins=[1900,1990,2000,2010,2015,2020,2025,3500,10000])

In [ ]:
#Conversión de códigos a parámetros categoricos
ACTORV_df.loc[ACTORV_df["CLASE_SINIESTRO"] == 1, "CLASE_SINIESTRO"] = 'CHOQUE'
ACTORV_df.loc[ACTORV_df["CLASE_SINIESTRO"] == 2, "CLASE_SINIESTRO"] = 'ATROPELLO'
ACTORV_df.loc[ACTORV_df["CLASE_SINIESTRO"] == 3, "CLASE_SINIESTRO"] = 'VOLVAMIENTO'
ACTORV_df.loc[ACTORV_df["CLASE_SINIESTRO"] == 4, "CLASE_SINIESTRO"] = 'CAIDA_OCC'

In [ ]:
#Conversión de códigos a parámetros categoricos
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 1, "CLASE_VEHICULO"] = 'AUTOMOVIL'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 2, "CLASE_VEHICULO"] = 'BUS'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 3, "CLASE_VEHICULO"] = 'BUSETA'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 4, "CLASE_VEHICULO"] = 'CAMION'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 5, "CLASE_VEHICULO"] = 'CAMIONETA'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 6, "CLASE_VEHICULO"] = 'CAMPERO'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 7, "CLASE_VEHICULO"] = 'MICROBUS'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 8, "CLASE_VEHICULO"] = 'TRACTOCAMION'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 9, "CLASE_VEHICULO"] = 'VOLQUETA'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 10, "CLASE_VEHICULO"] = 'MOTOCICLETA'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 13, "CLASE_VEHICULO"] = 'BICICLETA'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 17, "CLASE_VEHICULO"] = 'MOTOCICLO'
ACTORV_df.loc[ACTORV_df["CLASE_VEHICULO"] == 30, "CLASE_VEHICULO"] = 'MOD_PEATONAL'

In [ ]:
#Conversión de códigos a parámetros categoricos
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 1000, "CODIGO_CAUSA"] = 'OTRAS'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 98, "CODIGO_CAUSA"] = 'TRANSIT_ENTRE_VEH'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 102, "CODIGO_CAUSA"] = 'ADELANTAR_DER'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 103, "CODIGO_CAUSA"] = 'ADELANTAR_CERR'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 104, "CODIGO_CAUSA"] = 'ADELANTAR_INVA'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 106, "CODIGO_CAUSA"] = 'ADELANTAR_INVA'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 112, "CODIGO_CAUSA"] = 'DESOBEDECER_SENAL'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 114, "CODIGO_CAUSA"] = 'EMBRIAGUEZ'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 115, "CODIGO_CAUSA"] = 'EMBRIAGUEZ'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 119, "CODIGO_CAUSA"] = 'FRENAR_BRUSC'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 121, "CODIGO_CAUSA"] = 'NO_MANTENER_DIST'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 122, "CODIGO_CAUSA"] = 'NO_SENAL_GIRO'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 123, "CODIGO_CAUSA"] = 'NO_RESP_PREL'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 127, "CODIGO_CAUSA"] = 'CONTRAVIA'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 132, "CODIGO_CAUSA"] = 'NO_RESP_PREL'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 134, "CODIGO_CAUSA"] = 'REVERSO_IMPRU'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 139, "CODIGO_CAUSA"] = 'IMPERICIA_MANEJO'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 141, "CODIGO_CAUSA"] = 'VEHICLO_MAL_EST'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 142, "CODIGO_CAUSA"] = 'SEMAFORO_ROJO'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 143, "CODIGO_CAUSA"] = 'ARRANCAR_SIN_PREC'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 145, "CODIGO_CAUSA"] = 'ARRANCAR_SIN_PREC'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 157, "CODIGO_CAUSA"] = 'NO_DETALLE'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 306, "CODIGO_CAUSA"] = 'HUECOS'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 409, "CODIGO_CAUSA"] = 'NO_OBSERVAR'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 411, "CODIGO_CAUSA"] = 'OTRAS'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 506, "CODIGO_CAUSA"] = 'OTRAS'
ACTORV_df.loc[ACTORV_df["CODIGO_CAUSA"] == 750, "CODIGO_CAUSA"] = 'MANT_MECANICO'

In [ ]:
#Conversión de códigos a parámetros categoricos
ACTORV_df.loc[ACTORV_df["SERVICIO_VEHICULO"] == 1, "SERVICIO_VEHICULO"] = 'OFICIAL'
ACTORV_df.loc[ACTORV_df["SERVICIO_VEHICULO"] == 2, "SERVICIO_VEHICULO"] = 'PUBLICO'
ACTORV_df.loc[ACTORV_df["SERVICIO_VEHICULO"] == 3, "SERVICIO_VEHICULO"] = 'PARTICULAR'
ACTORV_df.loc[ACTORV_df["SERVICIO_VEHICULO"] == 5, "SERVICIO_VEHICULO"] = 'SER_BICICLETA'
ACTORV_df.loc[ACTORV_df["SERVICIO_VEHICULO"] == 6, "SERVICIO_VEHICULO"] = 'SER_PEATONAL'

In [ ]:
#Parametrización de la variable respuesta en códigos para los siguientes modelos.
ACTORV_df.loc[ACTORV_df["GRAVEDAD"] == 1, "GRAVEDAD"] = 0
ACTORV_df.loc[ACTORV_df["GRAVEDAD"] == 2, "GRAVEDAD"] = 1
ACTORV_df.loc[ACTORV_df["GRAVEDAD"] == 3, "GRAVEDAD"] = 2

In [ ]:
#Eliminación de valores faltantes o nulos
ACTOR_ML=ACTORV_df[['GRAVEDAD', 'TIPO_ACTOR','GENERO','CLASE_SINIESTRO','CLASE_VEHICULO','SERVICIO_VEHICULO','CODIGO_CAUSA','MODELO_CAT','EDAD']]
ACTOR_ML = ACTOR_ML.dropna(axis=0)
ACTOR_ML.head(10)

,GRAVEDAD,TIPO_ACTOR,GENERO,CLASE_SINIESTRO,CLASE_VEHICULO,SERVICIO_VEHICULO,CODIGO_CAUSA,MODELO_CAT,EDAD
0,2,CONDUCTOR,MASCULINO,CHOQUE,CAMION,PUBLICO,NO_MANTENER_DIST,"(1990, 2000]",24
1,2,CONDUCTOR,MASCULINO,CHOQUE,CAMIONETA,PARTICULAR,NO_MANTENER_DIST,"(2020, 2025]",54
2,1,CONDUCTOR,MASCULINO,CHOQUE,BUS,PUBLICO,NO_MANTENER_DIST,"(2015, 2020]",51
3,1,CONDUCTOR,MASCULINO,CHOQUE,BUS,PUBLICO,OTRAS,"(2015, 2020]",51
4,1,CONDUCTOR,MASCULINO,CHOQUE,BUS,PUBLICO,NO_MANTENER_DIST,"(2010, 2015]",47
5,1,CONDUCTOR,MASCULINO,CHOQUE,BUS,PUBLICO,OTRAS,"(2010, 2015]",47
6,1,CONDUCTOR,MASCULINO,CHOQUE,BUS,PUBLICO,NO_MANTENER_DIST,"(2010, 2015]",49
7,1,PASAJERO,MASCULINO,CHOQUE,BUS,PUBLICO,NO_MANTENER_DIST,"(2010, 2015]",21
8,1,CONDUCTOR,MASCULINO,CHOQUE,BUS,PUBLICO,OTRAS,"(2010, 2015]",49
9,1,PASAJERO,MASCULINO,CHOQUE,BUS,PUBLICO,OTRAS,"(2010, 2015]",21


### 3. ESTADÍSTICA DESCRIPTIVA
------------
Para el mayor entendimiento de la problemática resulta idóneo la aplicación de técnicas de estadística descriptiva, así bien, para las variables categóricas se aplicaran tablas pivot que permiten el adecuado estudio de este tipo de variables.

In [ ]:
PIV_ACTOR = ACTOR_ML.pivot_table(ACTOR_ML, index=['GRAVEDAD','GENERO'],columns=['TIPO_ACTOR'],
                    aggfunc={'TIPO_ACTOR': 'count'}).fillna(0).astype(int)
PIV_ACTOR_DF=PIV_ACTOR.reset_index()
PIV_VEHICULO = ACTOR_ML.pivot_table(ACTOR_ML, index=['GRAVEDAD','GENERO'],columns=['CLASE_VEHICULO'],
                    aggfunc={'CLASE_VEHICULO': 'count'}).fillna(0).astype(int)
PIV_VEHICULO_DF=(PIV_VEHICULO .reset_index()).iloc[:,2:]

#PIV_EDAD = ACTOR_ML.pivot_table(ACTOR_ML, index=['GRAVEDAD','GENERO'],columns=['EDAD_CAT'],
#                    aggfunc={'EDAD_CAT': 'count'}).fillna(0).astype(int)
#PIV_EDAD_DF=(PIV_EDAD.reset_index()).iloc[:,2:]
df_CONCAT=pd.concat([PIV_ACTOR_DF,PIV_VEHICULO_DF], axis=1)
df_CONCAT

GRAVEDAD     GENERO  TIPO_ACTOR                                           \
                      ACOMPANANTE CICLISTA CONDUCTOR MOTOCICLISTA PASAJERO   
0        0   FEMENINO         157       32        38           55       47   
1        0  MASCULINO         134      250       799          724       44   
2        1   FEMENINO        4314     1731      2209         2076     3254   
3        1  MASCULINO        2102     4927     21181        16734     1345   
4        2   FEMENINO           0       76      8912          435        0   
5        2  MASCULINO           0      490     66062         4767        0   

         CLASE_VEHICULO            ...                                  \
  PEATON      AUTOMOVIL BICICLETA  ... BUSETA CAMION CAMIONETA CAMPERO   
0    110             72        32  ...      1      1        20       6   
1    268            256       250  ...     11    131       118      17   
2   2370           3441      1731  ...    140     45       921     236   
3   2671          11651      4927  ...    303    982      4107     848   
4      0           6008        76  ...      6     57      2058     619   
5      0          30435       490  ...    745   5822     13805    3070   

                                                                     
  MICROBUS MOD_PEATONAL MOTOCICLETA MOTOCICLO TRACTOCAMION VOLQUETA  
0        6          110         179         0            0        0  
1       30          268         814         3           66       50  
2      247         2370        5041        36            8       10  
3      678         2671       18031        62          228      193  
4       48            0         434         2            9        3  
5     1409            0        4765        11         1842      965  

[6 rows x 21 columns]

### 4. PREPROCESAMIENTO INFORMACION
------------------
Para la aplicación de técnicas de Machine Learning, es importante el tratamiento adecuado de los datos que permita la posterior aplicación de las variables, así bien, en este caso al ser variables categoría, se aplicara la metodología de *“OneHotEncoder”* para el tratamiento de variables categóricas.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
# Identificación de las variables exploratorias y respuesta para la base de datos de actores viales
X_ACTORV = ACTOR_ML.drop(['GRAVEDAD'], axis=1)
y_ACTORV  = ACTOR_ML.GRAVEDAD

print(f'El shape de X_ACTORV es: {X_ACTORV.shape}')
print(f'El shape de y_ACTORV es: {y_ACTORV.shape}')
X_ACTORV.info()

El shape de X_ACTORV es: (148314, 8)
El shape de y_ACTORV es: (148314,)
<class 'pandas.core.frame.DataFrame'>
Index: 148314 entries, 0 to 158837
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   TIPO_ACTOR         148314 non-null  object  
 1   GENERO             148314 non-null  object  
 2   CLASE_SINIESTRO    148314 non-null  object  
 3   CLASE_VEHICULO     148314 non-null  object  
 4   SERVICIO_VEHICULO  148314 non-null  object  
 5   CODIGO_CAUSA       148314 non-null  object  
 6   MODELO_CAT         148314 non-null  category
 7   EDAD               148314 non-null  Int64   
dtypes: Int64(1), category(1), object(6)
memory usage: 9.3+ MB


In [ ]:
#X_ACTORV.to_csv('X_ACTORV_2.csv', index=False,header=True)

In [ ]:
#Transformación de las variables categóricas
categoric=list(X_ACTORV.select_dtypes(['object','category']).columns)
numeric = list(X_ACTORV.select_dtypes(['int64']).columns)

tf = ColumnTransformer([('onehot', OneHotEncoder(), categoric),
                        ('scaler', StandardScaler(), numeric)])

X_ACTORV_prepr = tf.fit_transform(X_ACTORV)

print(f'El shape de X_ACTORV es: {X_ACTORV.shape}')
print(f'El shape de X_ACTORV_prepr es: {X_ACTORV_prepr.shape}')

El shape de X_ACTORV es: (148314, 8)
El shape de X_ACTORV_prepr es: (148314, 57)


In [ ]:
categoric

['TIPO_ACTOR',
 'GENERO',
 'CLASE_SINIESTRO',
 'CLASE_VEHICULO',
 'SERVICIO_VEHICULO',
 'CODIGO_CAUSA',
 'MODELO_CAT']

In [ ]:
numeric

['EDAD']

In [ ]:
#Nombre con las diferentes Categorias
onehot_categories = tf.named_transformers_['onehot'].get_feature_names_out()
scaler_categories = tf.named_transformers_['scaler'].get_feature_names_out()
ACTOR_NAMES=np.concatenate((onehot_categories, scaler_categories), axis=0)
ACTOR_NAMES

array(['TIPO_ACTOR_ACOMPANANTE', 'TIPO_ACTOR_CICLISTA',
       'TIPO_ACTOR_CONDUCTOR', 'TIPO_ACTOR_MOTOCICLISTA',
       'TIPO_ACTOR_PASAJERO', 'TIPO_ACTOR_PEATON', 'GENERO_FEMENINO',
       'GENERO_MASCULINO', 'CLASE_SINIESTRO_ATROPELLO',
       'CLASE_SINIESTRO_CAIDA_OCC', 'CLASE_SINIESTRO_CHOQUE',
       'CLASE_SINIESTRO_VOLVAMIENTO', 'CLASE_VEHICULO_AUTOMOVIL',
       'CLASE_VEHICULO_BICICLETA', 'CLASE_VEHICULO_BUS',
       'CLASE_VEHICULO_BUSETA', 'CLASE_VEHICULO_CAMION',
       'CLASE_VEHICULO_CAMIONETA', 'CLASE_VEHICULO_CAMPERO',
       'CLASE_VEHICULO_MICROBUS', 'CLASE_VEHICULO_MOD_PEATONAL',
       'CLASE_VEHICULO_MOTOCICLETA', 'CLASE_VEHICULO_MOTOCICLO',
       'CLASE_VEHICULO_TRACTOCAMION', 'CLASE_VEHICULO_VOLQUETA',
       'SERVICIO_VEHICULO_OFICIAL', 'SERVICIO_VEHICULO_PARTICULAR',
       'SERVICIO_VEHICULO_PUBLICO', 'SERVICIO_VEHICULO_SER_BICICLETA',
       'SERVICIO_VEHICULO_SER_PEATONAL', 'CODIGO_CAUSA_ADELANTAR_CERR',
       'CODIGO_CAUSA_ADELANTAR_DER', 'CODIGO_CAUSA_

### 5. DEFINICIÓN DE BASES DE ENTRENAMIENTO Y PRUEBA
------------
Para los ejercicios de Machine Learning, es relevante la adecuada separación de los conjuntos de entrenamiento y pruebas, así bien se aplican funciones de *sklearn* para la separación de los datos.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_ACTORV_prepr,
                                                    y_ACTORV,
                                                    test_size=0.3,
                                                    stratify=y_ACTORV,
                                                    random_state=42)

In [ ]:
print(f'Número de muestras en entrenamiento: {X_train.shape[0]}')
print(f'Número de muestras en prueba: {X_test.shape[0]}')
print(f'Número de características: {X_train.shape[1]}')
print(f'Distribución de clases en entrenamiento: {np.bincount(y_train)}')
print(f'Distribución de clases en prueba: {np.bincount(y_test)}')

Número de muestras en entrenamiento: 103819
Número de muestras en prueba: 44495
Número de características: 57
Distribución de clases en entrenamiento: [ 1861 45439 56519]
Distribución de clases en prueba: [  797 19475 24223]


In [ ]:
gravedad_names=['MUERTO', 'HERIDO', 'DANOS']


## IV APLICACION DE ALGORITMOS DE ML
--------------
En el siguiente apartado se aplicarán diferentes algoritmos de aprendizaje para clasificación de parametros.

En este apartado, primero se aplican algoritmos por validación cruzada para la identificación de los modelos de mayor desempeño y posteriormente se prueba el desempeño del mejor modelo.

#### A) Algoritmo: K-vecinos más cercanos (KNN)



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

##### KNN GridSearchCV

In [ ]:
#parámetros básicos para la aplicación del CV-grid
knn = KNeighborsClassifier()
k_range = list(range(2, 5))
param_grid = dict(n_neighbors=k_range)

In [ ]:
# Aplicación de CV-Grid para identificación de Hyperparametros
grid_knn = GridSearchCV(knn,
                   param_grid=param_grid,
                   verbose=3,
                   cv=5,
                   n_jobs=-1,
                   scoring='accuracy',
                   return_train_score=True
                   )
grid_knn.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [2, 3, 4]}, return_train_score=True,
             scoring='accuracy', verbose=3)

In [ ]:
#Cálculo de los resultados del Grid
pd.DataFrame(grid_knn.cv_results_).sort_values(by='rank_test_score', ascending=True).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,0.0550,0.026704,232.942534,68.286042,3,{'n_neighbors': 3},0.765941,0.768349,0.774465,0.774176,...,0.772103,0.004289,1,0.832593,0.834989,0.831882,0.832918,0.831523,0.832781,0.001210
2,0.0220,0.002828,149.442600,1.155266,4,{'n_neighbors': 4},0.759777,0.759391,0.763870,0.764496,...,0.763194,0.003339,2,0.822599,0.823888,0.820601,0.821432,0.820603,0.821825,0.001265
0,0.0606,0.012224,292.086824,0.380473,2,{'n_neighbors': 2},0.690089,0.693026,0.700058,0.698420,...,0.697300,0.005235,3,0.786684,0.790067,0.787310,0.789212,0.787998,0.788254,0.001235


In [ ]:
resut_KNN= pd.DataFrame(grid_knn.cv_results_).sort_values(by='rank_test_score', ascending=True)
resut_KNN.to_csv('C:\\sqldeveloper\\sqldeveloper\\bin\\resut_KNN_cv5.csv', index=False,header=True)

In [ ]:
# Mejores parámetros identificados.
print(grid_knn.best_params_)
n_neighbors=list(grid_knn.best_params_.values())[0]

{'n_neighbors': 3}


In [ ]:
# Puntaje de la mejor combinación de parámetros.
print(grid_knn.best_score_)

0.7721034250156751


##### KNN Best Model

In [ ]:
#Caculo del modelo optimizado previamente identificado, con los mejores hiperparametros
KNN_cl = KNeighborsClassifier(n_neighbors=n_neighbors)

In [ ]:
KNN_fit=KNN_cl.fit(X_train, y_train)

In [ ]:
print(f'Error entrenamiento: {1 - KNN_fit.score(X_train, y_train)}')
print(f'Error generalización: {1 - KNN_fit.score(X_test, y_test)}')

Error entrenamiento: 0.17019042757106118
Error generalización: 0.22618271715923133


In [ ]:
#Estimación de la predicción para el mejor modelo
KNN_pred = KNN_fit.predict(X_test)

In [ ]:
# Generación de la matriz de confusión.
KNN_mat = confusion_matrix(y_test, KNN_pred)
list_confusion_matrix(KNN_mat, ['MUERTO', 'HERIDO', 'DANOS'])

Valor predicho              
                          MUERTO HERIDO  DANOS
Valor real MUERTO             71    564    162
           HERIDO            322  14013   5140
           DANOS             163   3713  20347

In [ ]:
#Estimación de los estadísticos de desempeño del modelo.
print(f'Orden de las etiquetas: {gravedad_names}')
print(f'Precisión por clase: \t{precision_score(y_test, KNN_pred, average=None)}')
print(f'Recall por clase: \t{recall_score(y_test, KNN_pred, average=None)}')
print(f'F1 por clase: \t{f1_score(y_test, KNN_pred, average=None)}')

Orden de las etiquetas: ['MUERTO', 'HERIDO', 'DANOS']
Precisión por clase: 	[0.12769784 0.76615637 0.79328629]
Recall por clase: 	[0.08908407 0.71953787 0.83998679]
F1 por clase: 	[0.10495196 0.74211572 0.81596888]


In [ ]:
#Estimación de los estadísticos contemplado los errores en el modelo
print(f"Precisión micro: {precision_score(y_test, KNN_pred, average='micro'):.4f}")
print(f"Recall micro: {recall_score(y_test, KNN_pred, average='micro'):.4f}")
print(f"F1 micro: {f1_score(y_test, KNN_pred, average='micro'):.4f}")

Precisión micro: 0.7738
Recall micro: 0.7738
F1 micro: 0.7738


#### B)  Algoritmo: RAMDOM FORREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from IPython.display import SVG
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

##### RANDOM FOREST GridSearchCV

In [ ]:
# Clasificador por Random Forrest
rfc=RandomForestClassifier(random_state=42)

In [ ]:
#parámetros básicos para la aplicación del CV-grid
param_grid = {
    'bootstrap': [True, False],
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
# Aplicación de CV-Grid para identificación de Hiperparametros
grid_rfc = GridSearchCV(rfc,
                   param_grid=param_grid,
                   verbose=3,
                   cv=5,
                   n_jobs=-1,
                   return_train_score=True
                   )
grid_rfc.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 500]},
             return_train_score=True, verbose=3)

In [ ]:
#Cálculo de los resultados del Grid
pd.DataFrame(grid_rfc.cv_results_).sort_values(by='rank_test_score', ascending=True).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,18.253412,0.270828,1.090188,0.028044,True,gini,4,sqrt,500,"{'bootstrap': True, 'criterion': 'gini', 'max_...",...,0.808195,0.003694,1,0.809030,0.809163,0.807429,0.808199,0.806805,0.808125,0.000909
40,7.620834,0.082976,0.483358,0.013322,True,entropy,5,log2,200,"{'bootstrap': True, 'criterion': 'entropy', 'm...",...,0.808147,0.003699,2,0.809006,0.809163,0.807393,0.808248,0.806817,0.808125,0.000907
41,18.594999,0.207165,1.170200,0.020083,True,entropy,5,log2,500,"{'bootstrap': True, 'criterion': 'entropy', 'm...",...,0.808137,0.003638,3,0.809078,0.809199,0.807441,0.808320,0.806865,0.808181,0.000910
68,8.287798,0.100712,0.476601,0.020774,False,gini,5,sqrt,200,"{'bootstrap': False, 'criterion': 'gini', 'max...",...,0.808118,0.003737,4,0.808946,0.809139,0.807369,0.808187,0.806757,0.808079,0.000910
33,17.446103,0.484673,1.099000,0.029550,True,entropy,4,sqrt,500,"{'bootstrap': True, 'criterion': 'entropy', 'm...",...,0.808118,0.003653,5,0.809042,0.809235,0.807453,0.808248,0.806781,0.808152,0.000932


In [ ]:
resut_RFC= pd.DataFrame(grid_knn.cv_results_).sort_values(by='rank_test_score', ascending=True)
resut_RFC.to_csv('C:\\sqldeveloper\\sqldeveloper\\bin\\resut_RFC_cv5.csv', index=False,header=True)

In [ ]:
# Mejores parámetros identificados.
print(grid_rfc.best_params_)
bootstrap,criterion,max_depth,max_features,n_estimators = list(grid_rfc.best_params_.values())

{'bootstrap': True, 'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 500}


In [ ]:
# Puntaje de la mejor combinación de parámetros.
print(grid_rfc.best_score_)

0.8081950829746166


##### RANDOM FOREST Best Model

In [ ]:
#Caculo del modelo optimizado previamente identificado, con los mejores hiperparametros
rfc=RandomForestClassifier(random_state=42,bootstrap=bootstrap,
                           criterion=criterion,max_depth=max_depth,
                           max_features=max_features,n_estimators=n_estimators)


In [ ]:
rfcfit = rfc.fit(X_train, y_train)

In [ ]:
print(f'Error entrenamiento: {1 - rfcfit.score(X_train, y_train)}')
print(f'Error generalización: {1 - rfcfit.score(X_test, y_test)}')

Error entrenamiento: 0.1917953361138135
Error generalización: 0.19258343634116193


In [ ]:
#Estimación de la predicción para el mejor modelo
rfc_pred= rfcfit.predict(X_test)

In [ ]:
# Generación de la matriz de confusión.
rfc_mat = confusion_matrix(y_test, rfc_pred)
list_confusion_matrix(rfc_mat, ['MUERTO', 'HERIDO', 'DANOS'])

Valor predicho              
                          MUERTO HERIDO  DANOS
Valor real MUERTO              0    610    187
           HERIDO              0  13468   6007
           DANOS               0   1765  22458

In [ ]:
#Estimación de los estadísticos de desempeño del modelo.
print(f'Orden de las etiquetas: {gravedad_names}')
print(f'Precisión por clase: \t{precision_score(y_test, rfc_pred, average=None)}')
print(f'Recall por clase: \t{recall_score(y_test, rfc_pred, average=None)}')
print(f'F1 por clase: \t{f1_score(y_test, rfc_pred, average=None)}')

Orden de las etiquetas: ['MUERTO', 'HERIDO', 'DANOS']
Precisión por clase: 	[0.         0.85009152 0.78381963]
Recall por clase: 	[0.         0.69155327 0.92713537]
F1 por clase: 	[0.         0.76267059 0.84947518]


In [ ]:
print(f"Precisión micro: {precision_score(y_test, rfc_pred, average='micro'):.4f}")
print(f"Recall micro: {recall_score(y_test, rfc_pred, average='micro'):.4f}")
print(f"F1 micro: {f1_score(y_test, rfc_pred, average='micro'):.4f}")

Precisión micro: 0.8074
Recall micro: 0.8074
F1 micro: 0.8074


#### C)  Algoritmo: MAQUINA DE SOPORTE VECTORIAL


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from IPython.display import SVG
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

##### SVM GridSearchCV

In [ ]:
#Clasificador por máquina de soporte vectorial
SVM_CL=SVC(kernel='rbf',random_state=42)

In [ ]:
#parámetros básicos para la aplicación del CV-grid, deben estar en forma de diccionario.
param_grid = {'C': [0.1, 1, 10],
              'gamma': [1, 0.1, 0.01]}


In [ ]:
# Aplicación de CV-Grid para identificación de Hiperparametros
grid_SVM_clf = GridSearchCV(SVM_CL,
                   param_grid=param_grid,
                   verbose=3,
                   cv=5,
                   n_jobs=-1,
                   return_train_score=True
                   )

grid_SVM_clf.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5, estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01]},
             return_train_score=True, verbose=3)

In [ ]:
#Cálculo de los resultados del Grid
pd.DataFrame(grid_SVM_clf.cv_results_).sort_values(by='rank_test_score', ascending=True).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
7,5586.705925,603.665480,49.180998,13.784774,10,0.1,"{'C': 10, 'gamma': 0.1}",0.809526,0.809622,0.818628,...,0.814312,0.004390,1,0.819084,0.819457,0.817519,0.818783,0.817160,0.818400,0.000900
4,1860.762735,16.202166,63.408598,0.928881,1,0.1,"{'C': 1, 'gamma': 0.1}",0.808081,0.808996,0.816558,...,0.812703,0.003879,2,0.814797,0.814809,0.812763,0.813967,0.812283,0.813724,0.001038
0,1620.706597,82.395784,78.153601,0.219457,0.1,1,"{'C': 0.1, 'gamma': 1}",0.808467,0.808129,0.814053,...,0.811730,0.003234,3,0.817001,0.816929,0.815448,0.816832,0.814980,0.816238,0.000851
3,2260.350344,117.923752,73.650002,0.405598,1,1,"{'C': 1, 'gamma': 1}",0.806348,0.807744,0.814487,...,0.811701,0.004383,4,0.830064,0.830353,0.828102,0.828897,0.826936,0.828870,0.001262
8,1348.004341,101.767107,54.055000,8.711715,10,0.01,"{'C': 10, 'gamma': 0.01}",0.806396,0.807937,0.815065,...,0.811162,0.003867,5,0.812883,0.812100,0.810138,0.811751,0.810056,0.811386,0.001115


In [ ]:
resut_SVM= pd.DataFrame(grid_SVM_clf.cv_results_).sort_values(by='rank_test_score', ascending=True)
resut_SVM.to_csv('C:\\sqldeveloper\\sqldeveloper\\bin\\resut_SVM_cv5.csv', index=False,header=True)

In [ ]:
# Mejores parámetros identificados.
print(grid_SVM_clf.best_params_)
C,gamma = list(grid_SVM_clf.best_params_.values())

{'C': 10, 'gamma': 0.1}


In [ ]:
# Puntaje de la mejor combinación de parámetros.
print(grid_SVM_clf.best_score_)

0.8143115008319715


#####SVM Best Model

In [ ]:
#Caculo del modelo optimizado previamente identificado, con los mejores hiperparametros
SVM=SVC(kernel='rbf',random_state=42,C=C,gamma=gamma)

In [ ]:
SVM_fit = SVM.fit(X_train, y_train)

In [ ]:
print(f'Error entrenamiento: {1 - SVM_fit.score(X_train, y_train)}')
print(f'Error generalización: {1 - SVM_fit.score(X_test, y_test)}')

Error entrenamiento: 0.18198980918714303
Error generalización: 0.18674008315541069


In [ ]:
#Estimación de la predicción para el mejor modelo
SVM_pred= SVM_fit.predict(X_test)

In [ ]:
# Generación de la matriz de confusión.
SVM_mat = confusion_matrix(y_test, SVM_pred)
list_confusion_matrix(SVM_mat, ['MUERTO', 'HERIDO', 'DANOS'])

Valor predicho              
                          MUERTO HERIDO  DANOS
Valor real MUERTO              8    626    163
           HERIDO              4  14199   5272
           DANOS               3   2241  21979

In [ ]:
#Estimación de los estadísticos de desempeño del modelo.
print(f'Orden de las etiquetas: {gravedad_names}')
print(f'Precisión por clase: \t{precision_score(y_test, SVM_pred, average=None)}')
print(f'Recall por clase: \t{recall_score(y_test, SVM_pred, average=None)}')
print(f'F1 por clase: \t{f1_score(y_test, SVM_pred, average=None)}')

Orden de las etiquetas: ['MUERTO', 'HERIDO', 'DANOS']
Precisión por clase: 	[0.53333333 0.83200516 0.80174363]
Recall por clase: 	[0.01003764 0.72908858 0.90736077]
F1 por clase: 	[0.01970443 0.77715443 0.8512888 ]


In [ ]:
print(f"Precisión micro: {precision_score(y_test, SVM_pred, average='micro'):.4f}")
print(f"Recall micro: {recall_score(y_test, SVM_pred, average='micro'):.4f}")
print(f"F1 micro: {f1_score(y_test, SVM_pred, average='micro'):.4f}")

Precisión micro: 0.8133
Recall micro: 0.8133
F1 micro: 0.8133


#### D)  Algoritmo: REGRESION LOGISTICA

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
#Clasificador por REGRESION LOGISTICA
RLOG_CL=LogisticRegression(random_state=42)

In [ ]:
parameters = [  {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
                {'penalty':['none', 'elasticnet', 'l1', 'l2']}, 
                {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]

In [ ]:
grid_RLOG_clf = GridSearchCV(RLOG_CL,  
                            param_grid = parameters,
                            scoring = 'accuracy',
                            verbose=3,
                            cv=5,
                            n_jobs=-1,
                            return_train_score=True
                           )


grid_RLOG_clf.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid=[{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']},
                         {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100]}],
             return_train_score=True, scoring='accuracy', verbose=3)

In [ ]:
pd.DataFrame(grid_RLOG_clf.cv_results_).sort_values(by='rank_test_score', ascending=True).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_penalty,param_C,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
13,1.836799,0.202565,0.005400,0.002334,NaN,NaN,10,{'C': 10},0.807985,0.805240,...,0.810613,0.003772,1,0.811282,0.812257,0.809584,0.81074,0.809478,0.810668,0.001049
1,1.614799,0.093394,0.004200,0.000402,lbfgs,NaN,NaN,{'solver': 'lbfgs'},0.807985,0.805144,...,0.810565,0.003762,2,0.811270,0.812137,0.809716,0.81074,0.809442,0.810661,0.000993
8,1.471598,0.098262,0.004800,0.002137,NaN,l2,NaN,{'penalty': 'l2'},0.807985,0.805144,...,0.810565,0.003762,2,0.811270,0.812137,0.809716,0.81074,0.809442,0.810661,0.000993
12,1.612600,0.092332,0.006401,0.003383,NaN,NaN,1,{'C': 1},0.807985,0.805144,...,0.810565,0.003762,2,0.811270,0.812137,0.809716,0.81074,0.809442,0.810661,0.000993
0,2.772999,0.294179,0.005602,0.000799,newton-cg,NaN,NaN,{'solver': 'newton-cg'},0.807985,0.805144,...,0.810555,0.003784,5,0.811270,0.812257,0.809536,0.81068,0.809502,0.810649,0.001051


In [ ]:
resut_RLOG= pd.DataFrame(grid_RLOG_clf.cv_results_).sort_values(by='rank_test_score', ascending=True).head(5)
resut_RLOG.to_csv('C:\\sqldeveloper\\sqldeveloper\\bin\\resut_RLOG_cv5.csv', index=False,header=True)

In [ ]:
# Mejores parámetros identificados.
print(grid_RLOG_clf.best_params_)
C= list(grid_RLOG_clf.best_params_.values())[0]

{'C': 10}


In [ ]:
# Puntaje de la mejor combinación de parámetros.
print(grid_RLOG_clf.best_score_)

0.8106127479203542


In [ ]:
#Caculo del modelo optimizado previamente identificado, con los mejores hiperparametros
RLOG=LogisticRegression(random_state=42,C=C)

In [ ]:
RLOG_fit = RLOG.fit(X_train, y_train)

In [ ]:
print(f'Error entrenamiento: {1 - RLOG_fit.score(X_train, y_train)}')
print(f'Error generalización: {1 - RLOG_fit.score(X_test, y_test)}')

Error entrenamiento: 0.1896184706074996
Error generalización: 0.1900213507135633


In [ ]:
RLOG_pred= RLOG_fit.predict(X_test)

In [ ]:
# Generación de la matriz de confusión.
RLOG_mat = confusion_matrix(y_test, RLOG_pred)
list_confusion_matrix(RLOG_mat, ['MUERTO', 'HERIDO', 'DANOS'])

Valor predicho              
                          MUERTO HERIDO  DANOS
Valor real MUERTO              0    620    177
           HERIDO              0  13941   5534
           DANOS               0   2124  22099

In [ ]:
#Estimación de los estadísticos de desempeño del modelo.
print(f'Orden de las etiquetas: {gravedad_names}')
print(f'Precisión por clase: \t{precision_score(y_test, RLOG_pred, average=None)}')
print(f'Recall por clase: \t{recall_score(y_test, RLOG_pred, average=None)}')
print(f'F1 por clase: \t{f1_score(y_test, RLOG_pred, average=None)}')

Orden de las etiquetas: ['MUERTO', 'HERIDO', 'DANOS']
Precisión por clase: 	[0.         0.83554091 0.79464222]
Recall por clase: 	[0.         0.71584082 0.91231474]
F1 por clase: 	[0.         0.77107301 0.84942248]


In [ ]:
print(f"Precisión micro: {precision_score(y_test, RLOG_pred, average='micro'):.4f}")
print(f"Recall micro: {recall_score(y_test, RLOG_pred, average='micro'):.4f}")
print(f"F1 micro: {f1_score(y_test, RLOG_pred, average='micro'):.4f}")

Precisión micro: 0.8100
Recall micro: 0.8100
F1 micro: 0.8100


#### E)  Algoritmo: REGRESION RED NEURONAL

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
#Clasificador por MLPClassifier
MLP_CL=MLPClassifier(random_state=42,max_iter=1000,tol=1e-4)

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive', 'invscaling'],
}

In [ ]:
# Aplicación de CV-Grid para identificación de Hiperparametros
grid_MLP_clf = GridSearchCV(MLP_CL,
                   param_grid=parameter_space,
                   verbose=10,
                   cv=5,
                   n_jobs=-1,
                   return_train_score=True
                   )

grid_MLP_clf.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=1000, random_state=42),
             n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50),
                                                (100,)],
                         'learning_rate': ['constant', 'adaptive',
                                           'invscaling'],
                         'solver': ['sgd', 'adam', 'lbfgs']},
             return_train_score=True, verbose=10)

In [ ]:
pd.DataFrame(grid_MLP_clf.cv_results_).sort_values(by='rank_test_score', ascending=True).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_alpha,param_hidden_layer_sizes,param_learning_rate,param_solver,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
43,104.986132,18.286975,0.160493,0.041478,tanh,0.05,"(50, 100, 50)",invscaling,adam,"{'activation': 'tanh', 'alpha': 0.05, 'hidden_...",...,0.813637,0.004094,1,0.815520,0.815700,0.813702,0.814207,0.813523,0.814531,0.000911
40,123.066968,34.805852,0.126778,0.024428,tanh,0.05,"(50, 100, 50)",adaptive,adam,"{'activation': 'tanh', 'alpha': 0.05, 'hidden_...",...,0.813637,0.004094,1,0.815520,0.815700,0.813702,0.814207,0.813523,0.814531,0.000911
37,113.937116,18.659064,0.156061,0.027489,tanh,0.05,"(50, 100, 50)",constant,adam,"{'activation': 'tanh', 'alpha': 0.05, 'hidden_...",...,0.813637,0.004094,1,0.815520,0.815700,0.813702,0.814207,0.813523,0.814531,0.000911
90,869.634542,160.498833,0.161462,0.024303,relu,0.05,"(50, 100, 50)",constant,sgd,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",...,0.813531,0.003972,4,0.818554,0.817422,0.814797,0.816327,0.815016,0.816423,0.001426
106,114.317879,24.786911,0.229050,0.094434,relu,0.05,"(100,)",invscaling,adam,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",...,0.813531,0.003667,5,0.817193,0.817290,0.816062,0.815953,0.814908,0.816281,0.000882


In [ ]:
resut_MLP= pd.DataFrame(grid_MLP_clf.cv_results_).sort_values(by='rank_test_score', ascending=True)
resut_MLP.to_csv('C:\\sqldeveloper\\sqldeveloper\\bin\\resut_MLP.csv', index=False,header=True)

In [ ]:
# Mejores parámetros identificados.
print(grid_MLP_clf.best_params_)
activation, alpha, hidden_layer_sizes, learning_rate, solver= list(grid_MLP_clf.best_params_.values())

{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': 'adam'}


In [ ]:
# Puntaje de la mejor combinación de parámetros.
print(grid_MLP_clf.best_score_)

0.8136372462785163


In [ ]:
#Caculo del modelo optimizado previamente identificado, con los mejores hiperparametros
MLP=MLPClassifier(random_state=42,max_iter=1000,tol=1e-4,
                  activation= activation,
                  alpha= alpha,
                  hidden_layer_sizes= hidden_layer_sizes,
                  learning_rate= learning_rate,
                  solver= solver
                  )

In [ ]:
MLP_fit = MLP.fit(X_train, y_train)

In [ ]:
print(f'Error entrenamiento: {1 - MLP_fit.score(X_train, y_train)}')
print(f'Error generalización: {1 - MLP_fit.score(X_test, y_test)}')

Error entrenamiento: 0.18793284466234506
Error generalización: 0.18934711765366896


In [ ]:
MLP_pred= MLP_fit.predict(X_test)

In [ ]:
# Generación de la matriz de confusión.
MLP_mat = confusion_matrix(y_test, MLP_pred)
list_confusion_matrix(MLP_mat, ['MUERTO', 'HERIDO', 'DANOS'])

Valor predicho              
                          MUERTO HERIDO  DANOS
Valor real MUERTO              0    624    173
           HERIDO              0  13784   5691
           DANOS               0   1937  22286

In [ ]:
#Estimación de los estadísticos de desempeño del modelo.
print(f'Orden de las etiquetas: {gravedad_names}')
print(f'Precisión por clase: \t{precision_score(y_test, MLP_pred, average=None)}')
print(f'Recall por clase: \t{recall_score(y_test, MLP_pred, average=None)}')
print(f'F1 por clase: \t{f1_score(y_test, MLP_pred, average=None)}')

Orden de las etiquetas: ['MUERTO', 'HERIDO', 'DANOS']
Precisión por clase: 	[0.         0.843316   0.79168739]
Recall por clase: 	[0.         0.7077792  0.92003468]
F1 por clase: 	[0.         0.76962591 0.8510492 ]


In [ ]:
print(f"Precisión micro: {accuracy_score(y_test, MLP_pred):.4f}")
print(f"Precisión micro: {precision_score(y_test, MLP_pred, average='macro'):.4f}")
print(f"Recall micro: {recall_score(y_test, MLP_pred, average='macro'):.4f}")
print(f"F1 micro: {f1_score(y_test, MLP_pred, average='macro'):.4f}")

Precisión micro: 0.8107
Precisión micro: 0.5450
Recall micro: 0.5426
F1 micro: 0.5402


## v.	CONCLUSIONES
-----------------
Los resultados de la aplicación de algoritmos de clasificación de aprendizaje de máquina presentaron una buena estimación para identificar los niveles de severidad en los siniestros viales registrados en la ciudad de Bogotá, Colombia. La exactitud promedio para el control de pruebas con los diferentes algoritmos estuvo por encima del 80%, siendo aceptable para todos los modelos aplicados. La investigación mostró que la aplicación de algoritmos más complejos de estimar como lo son SVM y MLP presentó levemente un mejor desempeño sin que este resultara ser mayormente significativo respecto a metodologías como LOGREG, RF, KNN, siendo cualquiera una herramienta deseable para la estimación de la severidad de los accidentes en la ciudad de Bogotá. De manera que, entidades encargadas de la gestión de la seguridad vial y profesionales pueden aplicar metodologías basadas en clasificadores para establecer medidas preventivas y correctivas en el estudio de la siniestralidad, de manera que, la implementación de enfoques basados en aprendizaje de máquina resulta ser pertinente en función de los resultados.

Al analizar los resultados de la predicción para cada uno de los niveles de severidad, se evidencia que los clasificadores resultan ser particularmente eficiente en la estimación de eventos con heridos y solo daños teniendo un alto rendimiento en las diferentes métricas de desempeño, mientras que para el caso de los siniestros con fatalidades, las técnicas empleadas no presentaban un buen rendimiento, siendo KNN la única capaz de representar de manera parcial los eventos con fatalidad, esto se puede dar debido a la diferencia tan alta en la cantidad de eventos con esta severidad, así como a la necesidad de incorporar otras variables que pudieran estar relacionadas con el siniestro como lo son las condiciones climáticas, las características de la vía, entre otras.

Así bien, la investigación muestra la pertinencia en la implementación de enfoques basados en aprendizaje de máquina en el estudio de la severidad en la siniestralidad en ciudades como Bogotá, donde la aplicación de algoritmos de clasificación tienen un desempeño deseable en la estimación de la severidad para la estimación de heridos y daños, mientras para el caso de las fatalidades es menos técnicas como KNN presentan resultados alentadores en la estimación de siniestros mortales que pueden ser implementados en el estudio sistemático de la siniestralidad.

